# BioSTEAM User Guide Tutorial - Beginner's Guide
this runs in Ubuntu 22.04.2 LTS on WSL2, Python 3.10.12\
install BioSTEAM with 'pip install biosteam'\
see https://biosteam.readthedocs.io/en/latest/tutorial/index.html

## 8. Process specifications
see https://biosteam.readthedocs.io/en/latest/tutorial/Process_specifications.html

### 8.1. Analytical specifications
#### 8.1.1. Denature ethanol fuel in a bioethanol process
Vary the amount of denaturant to add according to the flow of bioethanol. The final bioethanol product must be 2 wt. % denaturant:

In [ ]:
from biosteam import settings, Chemical, Stream, units, main_flowsheet
import biosteam as bst; bst.nbtutorial()
# First name a new flowsheet
main_flowsheet.set_flowsheet('mix_ethanol_with_denaturant')

# Set the thermodynamic property package.
# In an actual process, much more chemicals
# would be defined, but here we keep it short.
settings.set_thermo(['Water', 'Ethanol', 'Octane'])

# Assume 40 million gal ethanol produced a year
# with 330 operating days
dehydrated_ethanol = Stream('dehydrated_ethanol', T=340,
                            Water=0.1, Ethanol=99.9, units='kg/hr')
operating_days_per_year = 330
dehydrated_ethanol.F_vol = 40e6 / operating_days_per_year
denaturant = Stream('denaturant', Octane=1)
M1 = units.Mixer('M1', ins=(dehydrated_ethanol, denaturant), outs='denatured_ethanol')

# Create the specification function.
@M1.add_specification(run=True) # Run mass and energy balance after specification function
def adjust_denaturant_flow():
    denaturant_over_ethanol_flow = 0.02 / 0.98 # A mass ratio
    denaturant.imass['Octane'] = denaturant_over_ethanol_flow * dehydrated_ethanol.F_mass

# Simulate, and check results.
M1.simulate()
M1.show(composition=True, flow='kg/hr')